<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/colab/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_path = '/content/drive/My Drive/dataset'

    # Install required packages
    !pip install pymatgen

else:
    dataset_path = 'dataset'

In [4]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN


In [5]:
# Read file
sample_df = pd.read_csv(f"{dataset_path}/high_MoS2/defects.csv")
sample_df.head()

,_id,descriptor_id,energy,fermi_level,homo_lumo_gap,lumo,homo,E_1
0,MoS2_Mo63Se2S125_1de54178-46e9-4e0c-a922-6e4c9...,c2e35d6d-0314-4d37-9385-16a73a4b5b57,-1371.421478,-1.658518,0.3771,-1.0636,-1.4407,-63.1033
1,MoS2_Mo63W1Se2S125_b8bd7ef6-1d0f-43b9-a34c-afb...,f537f6af-ab01-4c23-8cdb-ba719fb19571,-1391.335165,-1.420464,1.1336,-0.8537,-1.9873,-62.9331
2,MoS2_Mo63W1Se1S125_42a338e1-df26-431d-9f5a-7c9...,4821cae7-32c3-4f2e-a821-942d6aef19e6,-1385.453746,-1.530740,1.0928,-0.9063,-1.9991,-62.9609
3,MoS2_Mo63W1Se2S125_a947f260-838e-4154-b08c-477...,f537f6af-ab01-4c23-8cdb-ba719fb19571,-1391.358443,-1.328388,1.1440,-0.8570,-2.0010,-62.9335
4,MoS2_Mo63Se2S125_ae2c4329-dd70-4174-bc11-a26ba...,c2e35d6d-0314-4d37-9385-16a73a4b5b57,-1371.905267,-1.679023,0.1618,-1.3186,-1.4804,-63.1352


## Get combined file

In [ ]:
# To do this you need to handle the dataset size issue...


## Data to graphs

In [ ]:
# The combined csv with all the high and low datasets will be named: combined.csv
defects_combined_path = Path(f'{dataset_path}/combined/combined.csv')
defects_df = pd.read_csv(defects_combined_path)


In [ ]:
# Use functions in cif_to_graog to convert each row into raphs.

I think the formula that the model should get is, if you have material x missing(vacancy defect) in the lattice or replaced by material y(the substitutional defect), the energy change in that site has a specific value(formation energy). When this value is combined with other formation energies at different defect sites at distances z, you get a positive or negative change to the band gap of the host materials.
So attributes will be:
1. Defect type(vacncy or substitutional)
2. Atomic number change of the site.(from here you get the formation energy)
3. Get formation energy relation with distance. 